In [13]:
from e3nn.o3 import spherical_harmonics
from e3nn.o3 import wigner_3j
from torch import Tensor
import torch

In [11]:
r = torch.Tensor([1., 10., 20.])
r_ang = r/torch.linalg.vector_norm(r)


Y_1 = spherical_harmonics('1e', r_ang, normalize = 'component')
Y_2 = spherical_harmonics('2e', r_ang, normalize = 'component')
Y_3 = spherical_harmonics('2e', r_ang, normalize = 'component')

features = torch.concatenate([Y_1, Y_2, Y_3])

features

tensor([ 0.0218,  0.2183,  0.4366,  0.0436,  0.0218, -0.1265,  0.4361,  0.4351,
         0.0436,  0.0218, -0.1265,  0.4361,  0.4351])

In [18]:
l1, l2, l3 = 1, 2, 3
m1, m2, m3 = l1, l2, l3 # m1 = 0, m2 = 0, m3 = 0
C123 = wigner_3j(1, 2, 3)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000, -0.0690,  0.0000, -0.2673],
         [ 0.0000,  0.0000,  0.0000, -0.1690,  0.0000, -0.2182,  0.0000],
         [ 0.0000,  0.0000,  0.2390,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.2182,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.2673,  0.0000,  0.0690,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0000,  0.2182,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.2760,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.2928,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.2760,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2182,  0.0000]],

        [[ 0.2673,  0.0000, -0.0690,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.2182,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.2390,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.000

In [21]:
C123_000 = C123[m1, m2, m3]

C123_000

tensor(0.2928)

In [38]:
class weigner_3j_img_to_real():


    def __init__(self, l):
        self.l = l

    def __call__(self):
        matrix = torch.zeros((2*self.l + 1, 2*self.l + 1), dtype = torch.complex64)

        mult = 1
        for i in range(2*self.l + 1):
            
            if i < self.l:
                matrix[i, i] = 1.0j/2**(1/2.)
                matrix[2*self.l + 1 - i - 1, i] = 1/2**(1/2.)
            elif i == self.l:
                matrix[i, i] = 1.
            else:
                matrix[i, i] = (-1.0)*mult/2**(1/2.)
                matrix[2*self.l + 1 - i - 1, i] = (-1.0j)*mult/2**(1/2.)
                mult *= -1
        
        return matrix

In [41]:
weigner_3j_img_to_real(1)()*2**(1/2.)

tensor([[ 0.0000+1.0000j,  0.0000+0.0000j,  0.0000-1.0000j],
        [ 0.0000+0.0000j,  1.4142+0.0000j,  0.0000+0.0000j],
        [ 1.0000+0.0000j,  0.0000+0.0000j, -1.0000+0.0000j]])